In [9]:
# 📦 Імпорт необхідних бібліотек
import nltk

# 🛠️ Завантаження ресурсів
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('opinion_lexicon')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


True

In [11]:
!pip install openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 4.3 MB/s eta 0:00:00


In [12]:
import pandas as pd
from google.colab import files

# 🔼 Завантаження файлу з ПК
uploaded = files.upload()

# 📖 Зчитування Excel-файлу
df = pd.read_excel(next(iter(uploaded)))

# 🧾 Перейменування та вибір перших двох колонок: текст і мітка (стать або клас)
df = df.iloc[:, :2]
df.columns = ['text', 'label']

# 👁️ Перевірка
df.head()


Saving blog-gender-dataset.xlsx to blog-gender-dataset (3).xlsx


,text,label
0,Guest Demo: Eric Iverson’s Itty Bitty Search\...,M
1,Who moved my Cheese??? The world has been de...,M
2,Yesterday I attended a biweekly meeting of an...,M
3,Liam is nothing like Natalie. Natalie never w...,F
4,"In the EU we have browser choice, but few know...",M


In [15]:
def preprocess(text):
    if not isinstance(text, str):  # Перевірка типу
        return []

    # Приведення до нижнього регістру
    text = text.lower()

    # Видалення пунктуації
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Проста токенізація
    tokens = text.split()

    # Видалення стоп-слів
    tokens = [t for t in tokens if t not in stop_words]

    # Стемінг
    stemmed = [stemmer.stem(t) for t in tokens]

    return stemmed

# Застосування
df['tokens'] = df['text'].apply(preprocess)

# Перевірка
df[['text', 'tokens']].head()


,text,tokens
0,Guest Demo: Eric Iverson’s Itty Bitty Search\...,"[guest, demo, eric, iverson’, itti, bitti, sea..."
1,Who moved my Cheese??? The world has been de...,"[move, chees, world, develop, area, creat, dif..."
2,Yesterday I attended a biweekly meeting of an...,"[yesterday, attend, biweekli, meet, inform, uc..."
3,Liam is nothing like Natalie. Natalie never w...,"[liam, noth, like, natali, natali, never, went..."
4,"In the EU we have browser choice, but few know...","[eu, browser, choic, know, eu, tougher, monopo..."


In [16]:
from collections import Counter

# Розділимо тексти за класами
tokens_m = df[df['label'] == 'M']['tokens'].sum()  # список токенів чоловіків
tokens_f = df[df['label'] == 'F']['tokens'].sum()  # список токенів жінок

# Підрахунок частотності
freq_m = Counter(tokens_m)
freq_f = Counter(tokens_f)

# Перевірка: 10 найчастотніших слів у кожному класі
print("Top 10 words in class M:", freq_m.most_common(10))
print("Top 10 words in class F:", freq_f.most_common(10))


Top 10 words in class M: [('one', 2430), ('like', 2147), ('time', 1884), ('get', 1742), ('would', 1410), ('go', 1400), ('make', 1386), ('use', 1236), ('work', 1206), ('know', 1189)]
Top 10 words in class F: [('one', 2286), ('like', 2153), ('time', 1978), ('get', 1768), ('go', 1530), ('day', 1449), ('im', 1425), ('know', 1419), ('make', 1408), ('love', 1312)]


In [17]:
import numpy as np

# Створимо загальний словник із унікальних слів
vocab = list(set(df['tokens'].sum()))
vocab_size = len(vocab)

# Індекс для кожного слова
word2idx = {word: i for i, word in enumerate(vocab)}

# Функція перетворення списку токенів у вектор частотності
def text_to_vector(tokens):
    vec = np.zeros(vocab_size)
    for token in tokens:
        if token in word2idx:
            vec[word2idx[token]] += 1
    return vec

# Створимо матрицю ознак X
X = np.array(df['tokens'].apply(text_to_vector).tolist())

# Створимо вектор міток y: 'M' -> 1, 'F' -> 0
y = (df['label'] == 'M').astype(int).values


In [18]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_loss(y_true, y_pred):
    # логарифмічна функція втрат
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def gradient_descent(X, y, lr=0.01, epochs=1000):
    n_samples, n_features = X.shape
    weights = np.zeros(n_features)
    bias = 0

    for epoch in range(epochs):
        linear_model = np.dot(X, weights) + bias
        y_pred = sigmoid(linear_model)

        # Обчислення градієнтів
        dw = (1 / n_samples) * np.dot(X.T, (y_pred - y))
        db = (1 / n_samples) * np.sum(y_pred - y)

        # Оновлення ваг
        weights -= lr * dw
        bias -= lr * db

        if epoch % 100 == 0:
            loss = compute_loss(y, y_pred)
            print(f"Epoch {epoch}: Loss = {loss:.4f}")

    return weights, bias


In [19]:
# Навчання моделі
weights, bias = gradient_descent(X, y, lr=0.1, epochs=1000)

# Прогнозування
def predict(X, weights, bias, threshold=0.5):
    linear_model = np.dot(X, weights) + bias
    probs = sigmoid(linear_model)
    return (probs >= threshold).astype(int)

y_pred = predict(X, weights, bias)

# Оцінка точності
accuracy = np.mean(y_pred == y)
print(f"Accuracy on all data: {accuracy:.4f}")


Epoch 0: Loss = 0.6931
Epoch 100: Loss = 0.4948
Epoch 200: Loss = 0.4365
Epoch 300: Loss = 0.3980
Epoch 400: Loss = 0.3690
Epoch 500: Loss = 0.3458
Epoch 600: Loss = 0.3264
Epoch 700: Loss = 0.3098
Epoch 800: Loss = 0.2953
Epoch 900: Loss = 0.2825
Accuracy on all data: 0.9369


In [21]:
def predict_text(text, weights, bias, word2idx, vocab_size, threshold=0.5):
    # Використовуємо ту саму preprocess, що й раніше
    tokens = preprocess(text)

    # Функція text_to_vector: створює вектор частотності слів
    vec = np.zeros(vocab_size)
    for token in tokens:
        if token in word2idx:
            vec[word2idx[token]] += 1

    prob = sigmoid(np.dot(vec, weights) + bias)
    pred_class = 1 if prob >= threshold else 0
    return pred_class, prob

# Твіт для тесту
tweet = "This is an example tweet for sentiment analysis"

# Виклик функції
pred_class, prob = predict_text(tweet, weights, bias, word2idx, vocab_size)

print(f"Tweet: {tweet}")
print(f"Predicted class: {pred_class} (probability: {prob:.4f})")


Tweet: This is an example tweet for sentiment analysis
Predicted class: 0 (probability: 0.4829)


In [22]:
import json
import numpy as np

# Припустимо, weights — numpy-масив, bias — число або numpy-скаляр
def save_model(weights, bias, filename):
    model_data = {
        'weights': weights.tolist(),  # конвертуємо numpy масив у список
        'bias': float(bias)           # конвертуємо bias у float, якщо це numpy-тип
    }
    with open(filename, 'w') as f:
        json.dump(model_data, f)
    print(f"Model saved to {filename}")

def load_model(filename):
    with open(filename, 'r') as f:
        model_data = json.load(f)
    weights = np.array(model_data['weights'])
    bias = model_data['bias']
    print(f"Model loaded from {filename}")
    return weights, bias

# Використання
save_model(weights, bias, 'sentiment_model.json')

# Потім можна завантажити:
weights_loaded, bias_loaded = load_model('sentiment_model.json')


Model saved to sentiment_model.json
Model loaded from sentiment_model.json
